In [97]:
# Get data
using CSV

# import Stigler's data set
raw = CSV.read("stigler.csv")

# list of food
foods = raw[2:end,1]

# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
lower_bound = raw[1,2:end]

# data[f,i] is the amount of nutrient i contained in food f 
data = raw[2:end,2:end]

# Note: data is not a matrix, but a dataframe. 
# You can access individual elements as if it was a matrix:
a = data[1,1]   # get the first element 
println("The nutrient ($(nutrients[1])) in food ($(foods[1])) is $a")
typeof(a)       # will tell you that a is a floating point number

lower_bound = raw[1,2:end]
min_require = convert(Array, lower_bound)
nut_data = convert(Array, data)

The nutrient (Calories (1000)) in food (Wheat Flour (Enriched)) is 44.7


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[97]:26
└ @ Core In[97]:26
┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[97]:27
└ @ Core In[97]:27


77×9 Array{Union{Missing, Float64},2}:
 44.7  1411.0   2.0  365.0   0.0  55.4  33.3  441.0    0.0
 11.6   418.0   0.7   54.0   0.0   3.2   1.9   68.0    0.0
 11.8   377.0  14.4  175.0   0.0  14.4   8.8  114.0    0.0
 11.4   252.0   0.1   56.0   0.0  13.5   2.3   68.0    0.0
 36.0   897.0   1.7   99.0  30.9  17.4   7.9  106.0    0.0
 28.6   680.0   0.8   80.0   0.0  10.6   1.6  110.0    0.0
 21.2   460.0   0.6   41.0   0.0   2.0   4.8   60.0    0.0
 25.3   907.0   5.1  341.0   0.0  37.1   8.9   64.0    0.0
 15.0   488.0   2.5  115.0   0.0  13.8   8.5  126.0    0.0
 12.2   484.0   2.7  125.0   0.0  13.9   6.4  160.0    0.0
 12.4   439.0   1.1   82.0   0.0   9.9   3.0   66.0    0.0
  8.0   130.0   0.4   31.0  18.9   2.8   3.0   17.0    0.0
 12.5   288.0   0.5   50.0   0.0   0.0   0.0    0.0    0.0
  ⋮                                ⋮                      
 13.5   104.0   2.5  136.0   4.5   6.3   1.4   24.0  136.0
 20.0  1367.0   4.2  345.0   2.9  28.7  18.4  162.0    0.0
 17.4  1055.0   3

In [139]:
using JuMP, Clp

original = Model(solver=ClpSolver())
a = @variable(original, money_spent[1:size(foods,1)] >= 0) # Money spent on each food
b = @variable(original, food_cost[1:size(foods,1), 1:size(nutrients,1)]) # Nutrition cost

# Get the nutrition cost
for i in 1:size(foods,1)
    @constraint(original, food_cost[i,:] .== nut_data[i,:]*money_spent[i])
end

# Nutient needs to be >= zeor
for j in 1:size(nutrients,1)
    @constraint(original, sum(food_cost[:,j]) >= min_require[j])
end
    
@objective(original, Min, sum(money_spent))     # minimize the price

println("Normal diet formulation")

status = solve(original)

println(status)
println()
println("objective = ", getobjectivevalue(original) * 365 )
println()

for i in 1:size(foods,1)
    if(getvalue(money_spent[i]) > 0)
        println(foods[i]," makes up the optimal diet")
    end
end

for i in 1:size(foods,1)
    println(getvalue(food_cost[i,:]))
end

Normal diet formulation
Optimal

objective = 39.661731545466246

Wheat Flour (Enriched) makes up the optimal diet
Liver (Beef) makes up the optimal diet
Cabbage makes up the optimal diet
Spinach makes up the optimal diet
Navy Beans, Dried makes up the optimal diet
[1.3195, 41.6514, 0.0590381, 10.7745, -0.0, 1.63536, 0.982985, 13.0179, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, -0.0, 0.0, 0.0,

In [144]:
print(getdual(money_spent))

[0.0, 0.845028, 0.295598, 0.85928, 0.488905, 0.697754, 0.716618, 0.470793, 0.650135, 0.702682, 0.807327, 0.860545, 0.874567, 0.319325, 0.0436664, 0.878027, 0.778062, 0.829072, 0.234905, 0.865259, 0.698149, 0.909013, 0.823821, 0.626234, 0.923889, 0.938124, 0.934309, 0.896506, 0.919149, 6.93889e-17, 0.905858, 0.923735, 0.918808, 0.893023, 0.87305, 0.880944, 0.802597, 0.944935, 0.942666, 0.651791, 0.807805, 0.808368, 0.838166, 0.63215, 0.614812, 0.0, 0.654062, 0.828528, 0.79627, 0.554767, 0.335243, -3.33067e-16, 0.349929, 0.901664, 0.939103, 0.894879, 0.906897, 0.773733, 0.679953, 0.867687, 0.852611, 0.723622, 0.780271, 0.758248, 0.666784, 0.758024, 0.38925, 0.103139, 0.0, 0.916574, 0.962377, 0.633869, 0.833002, 0.694096, 0.855283, 0.47153, 0.924585]

# don't touch it!!!

In [145]:
using JuMP, Clp

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(foods,1)] >= 0) # money spent on each food
for i in 1:size(nutrients,1)
    @constraint(dualModel, sum(λ'*nut_data[:,i]) >= min_require[i])
end
@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println(getobjectivevalue(dualModel) * 365)
for i in 1:size(foods,1)
    if(getvalue(λ[i]) > 0)
        println(foods[i]," makes up the optimal diet cost ", round(getvalue(λ[i])*365)," per year")
    end
end

Optimal
39.66173154546625
Wheat Flour (Enriched) makes up the optimal diet cost 11.0 per year
Liver (Beef) makes up the optimal diet cost 1.0 per year
Cabbage makes up the optimal diet cost 4.0 per year
Spinach makes up the optimal diet cost 2.0 per year
Navy Beans, Dried makes up the optimal diet cost 22.0 per year


In [138]:
using JuMP, Clp

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(foods,1), 1:size(nutrients,1)] >= 0) # money spent on each nutrient
for i in 1:size(nutrients,1)
    #@constraint(dualModel, sum(λ'*nut_data[i,:]) >= min_require[i]) # ?
    @constraint(dualModel, sum(λ'*nut_data[:,i]) >= min_require[i])
end
@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println(getobjectivevalue(dualModel)*365)
for i in 1:size(nutrients,1)
    println(getvalue(λ[i])," dollars spent on ",nutrients[i])
end

Optimal
39.66173154546625
0.02951906167648827 dollars spent on Calories (1000)
0.0 dollars spent on Protein (g)
0.0 dollars spent on Calcium (g)
0.0 dollars spent on Iron (mg)
0.0 dollars spent on Vitamin A (1000 IU)
0.0 dollars spent on Thiamine (mg)
0.0 dollars spent on Riboflavin (mg)
0.0 dollars spent on Niacin (mg)
0.0 dollars spent on Ascorbic Acid (mg)


In [101]:
using JuMP, Clp

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(nutrients,1)] >= 0) # money spent on each nutrient per day
for i in 1:size(nutrients,1)
    @constraint(dualModel, sum(λ[i]*nut_data[:,i]) >= min_require[i])
end
@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println(getobjectivevalue(dualModel)*365)
for i in 1:size(nutrients,1)
    if(getvalue(λ[i]) > 0)
        println(getvalue(λ[i,:])," dollars spent on ",nutrients[i])
    end
end

Optimal
11.796000686327215
[0.00404476] dollars spent on Calories (1000)
[0.0034344] dollars spent on Protein (g)
[0.00485437] dollars spent on Calcium (g)
[0.00201613] dollars spent on Iron (mg)
[0.00198389] dollars spent on Vitamin A (1000 IU)
[0.00353635] dollars spent on Thiamine (mg)
[0.00590551] dollars spent on Riboflavin (mg)
[0.00358066] dollars spent on Niacin (mg)
[0.00296173] dollars spent on Ascorbic Acid (mg)


In [94]:
getvalue(sum(λ))

0.03231781009952662

In [89]:
size(min_require[1])

()

# Qustion b

In [39]:
# Get data
using CSV

# import Stigler's data set
raw = CSV.read("stiglerNew.csv")

# list of food
foods = raw[2:end,1]

# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
lower_bound = raw[1,2:end]

# data[f,i] is the amount of nutrient i contained in food f 
data = raw[2:end,2:end]

# Note: data is not a matrix, but a dataframe. 
# You can access individual elements as if it was a matrix:
a = data[1,1]   # get the first element 
println("The nutrient ($(nutrients[1])) in food ($(foods[1])) is $a")
typeof(a)       # will tell you that a is a floating point number

lower_bound = raw[1,2:end]
min_require = convert(Array, lower_bound)
nut_data = convert(Array, data)

using JuMP, Clp

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(foods,1)] >= 0) # money spent on each food
for i in 1:size(nutrients,1)
    @constraint(dualModel, sum(λ'*nut_data[:,i]) >= min_require[i])
end
@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println(getobjectivevalue(dualModel) * 365)
for i in 1:size(foods,1)
    if(getvalue(λ[i]) > 0)
        println(foods[i]," makes up the optimal diet cost ", round(getvalue(λ[i])*365)," per year")
    end
end

The nutrient (Calories (1000)) in food (Wheat Flour (Enriched)) is 44.7
Optimal
36.9982473745081
Wheat Flour (Enriched) makes up the optimal diet cost 24.0 per year
Liver (Beef) makes up the optimal diet cost 3.0 per year
Cabbage makes up the optimal diet cost 4.0 per year
Spinach makes up the optimal diet cost 1.0 per year
Calcium Supplements makes up the optimal diet cost 5.0 per year


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[39]:26
└ @ Core In[39]:26
┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[39]:27
└ @ Core In[39]:27
